# RNN

- RNN을 뉴런 단위로 시각화
![doc1](https://github.com/tenjumh/GraduateSchool/blob/master/Study/NLP_Natural%20Language%20Processing/image/RNN/rnn1.PNG?raw=True)
- 다양한 RNN 쓰임
    1. 스팸 메일 분류
    
![doc1](https://github.com/tenjumh/GraduateSchool/blob/master/Study/NLP_Natural%20Language%20Processing/image/RNN/rnn3.PNG?raw=True)

    2. 개체명 인식
    
![doc1](https://github.com/tenjumh/GraduateSchool/blob/master/Study/NLP_Natural%20Language%20Processing/image/RNN/rnn4.PNG?raw=True)

- RNN 수식 정의
![doc1](https://github.com/tenjumh/GraduateSchool/blob/master/Study/NLP_Natural%20Language%20Processing/image/RNN/rnn5.PNG?raw=True)
    1. 현 시점 t에서의 은닉 상태값을 h(t)라고 정의
    2. h(t)를 계산하기 위해서는 두 개의 가중치가 필요
    3. w(x)와 t-1시점의 은닉 상태값인 w(h)
    4. 식으로 표현하면
![doc1](https://github.com/tenjumh/GraduateSchool/blob/master/Study/NLP_Natural%20Language%20Processing/image/RNN/rnn6.PNG?raw=True)
    5. 자연어 처리에서 RNN의 입력 x(t)는 대부분 단어 벡터로 간주, 단어벡터의 차원을 d라고 하고, 은닉 상태의 크기를 D(h)라고 할 때,
![doc1](https://github.com/tenjumh/GraduateSchool/blob/master/Study/NLP_Natural%20Language%20Processing/image/RNN/rnn7.PNG?raw=True)
    6. 배치 크기가 1이고 d와 D(h) 두 값을 모두 4로 가정하면 그림은,
![doc1](https://github.com/tenjumh/GraduateSchool/blob/master/Study/NLP_Natural%20Language%20Processing/image/RNN/rnn8.PNG?raw=True)

# 케라스로 RNN 구현하기
- RNN층을 추가하는 코드
```
model.add(SimpleRNN(hidden_size)
```
- 추가 인자를 사용할 때
```
model.add(SimpleRNN(hidden_size, input_shape=(timesteps, input_dim)))
```
- 다른 표기
```
model.add(SimpleRNN(hidden_size, input_length=M, input_dim=N))  # M과 N은 정수
```
- hidden_size = 은닉상태의 크기 (메모리셀 = 다음 시점 메모리셀 = 출력층 보내는 값(output_dim) 크기 동일), 보통(128, 256, 512, 1024)
- timesteps = 입력 시퀀스 길이(input_length)
- input_dim = 입력의 크기

- RNN층은 input the 3D Tensor(batch_size, timesteps, input_dim)
- RNN층은 두 가지 종류 출력, <br>
    (1) 최종 시점의 은닉 상태만 리턴 - 2D 텐서(batch_size, output_dim) 리턴, <br>
    (2) 각 시점(time step)의 은닉 상태값들을 모아 전체 시퀀스를 리턴 - 3D 텐서(batch_size, timesteps, output_dim) 리턴<br>
- return_sequences 매개 변수에 True 설정하여 설정 가능

![doc1](https://github.com/tenjumh/GraduateSchool/blob/master/Study/NLP_Natural%20Language%20Processing/image/RNN/return_sequences.PNG?raw=True)
- 위 그림은 time step=3일 때, return_sequences=Ture와 False로 설정했을 때 차이
- True : 모든 time step에서 은닉 상태값을 출력 -- * many-to-many
- False or X : 마지막 time step의 은닉 상태값을 출력 -- * many-to-one

In [26]:
from keras.models import Sequential
from keras.layers import SimpleRNN

In [4]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10)) 동일
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


- output shape이 (None, 3) -> batch_size가 정해지지 않아서 None이며 3은 hidden_size이다.
- 그리고 return_sequences를 기재하지 않아 False로 진행되어 <b><h2> "2D" </h2></b>output이 출력된다.

In [5]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8, 2, 10)))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (8, 3)                    42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


- batch_size를 정의하면 (8, 3)임

In [10]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8, 2, 10), return_sequences=True))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_7 (SimpleRNN)     (8, 2, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


- return_sequences을 True로 하여 (batch_size, timesteps, output_dim) 크기의 <b><h2> "3D" </h2></b>텐서를 리턴한다.

# 파이썬으로 RNN 구현하기

메모리 셀에서 은닉 상태를 계산하는 식

![doc1](https://github.com/tenjumh/GraduateSchool/blob/master/Study/NLP_Natural%20Language%20Processing/image/RNN/rnn9.PNG?raw=True)

In [12]:
import numpy as np

timesteps = 10   # 이거슨 NLP에서 문장의 길이가 된다. 예 "나는 학교에 간다" 는 3이죠.
input_dim = 4   # 단어 벡터의 차원이다. 나는 = [1,0,0,0]   4차원
hidden_size = 8   # 메모리 셀의 용량

inputs = np.random.random((timesteps, input_dim))   # 입력에 해당하는 2D 텐서

hidden_state_t = np.zeros((hidden_size,))   # 초기 은닉 상태는 0벡터로 초기화
# hidden_stata_t의 상태 크기는 hidden_size로 은닉 상태를 만듬

In [13]:
print(hidden_state_t)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [14]:
# 가중치와 편향 정의

Wx = np.random.random((hidden_size, input_dim))    # (8, 4) : 4차원 벡터값이 8개의 은닉층으로 들어가서
Wh = np.random.random((hidden_size, hidden_size))     # (8, 8) : t-1 은닉층의 8개가 t 은닉층의 8개로 들어가서
b = np.random.random((hidden_size,))    # (8,)크기의 1D 텐서

In [18]:
print(Wx.shape)

(8, 4)


In [19]:
print(Wh.shape)

(8, 8)


In [20]:
print(b.shape)

(8,)


In [24]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs:  # 각 시점에 따라서 입력값이 입력
    output_t = np.tanh(np.dot(Wx, input_t) + np.dot(Wh, hidden_state_t) + b)
    # W(x)*X(t) + W(h)*H(t-1) + b
    total_hidden_states.append(list(output_t))   # 각 시점의 은닉 상태의 값을 계속 추적
    print(np.shape(total_hidden_states))  # 각 시점 t별 메모리 셀의 출력의 크기
    hidden_state_t = output_t
    
total_hidden_states = np.stack(total_hidden_states, axis = 0)
# 출력 시 값을 깔끔하게 해준다.

print(total_hidden_states)  # (timesteps, output_dim)의 크기

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.99998468 0.99998412 0.99997676 0.99998541 0.99997305 0.99995806
  0.99983635 0.99999874]
 [0.99998461 0.99997635 0.99996817 0.99998651 0.99996796 0.99996505
  0.99987546 0.99999881]
 [0.99999255 0.99998893 0.99999006 0.99999497 0.99998866 0.99997895
  0.99994091 0.99999953]
 [0.99995534 0.99996155 0.99997411 0.99998753 0.99996247 0.99996209
  0.99980169 0.99999882]
 [0.9999755  0.99996368 0.99997769 0.99999045 0.99997417 0.99998026
  0.99991953 0.99999916]
 [0.99999236 0.9999869  0.99999141 0.9999936  0.99999101 0.99999017
  0.99997015 0.99999951]
 [0.99996431 0.99997469 0.99996991 0.99996857 0.99996113 0.99996215
  0.99977801 0.99999772]
 [0.99997979 0.99998353 0.999986   0.99998814 0.99998068 0.99997195
  0.99987384 0.99999901]
 [0.99997934 0.99998384 0.99997714 0.99996414 0.99997372 0.99997385
  0.99986136 0.99999776]
 [0.99998529 0.99998412 0.99999196 0.99999081 0.99998978 0.99999008
  0.99995854 0.99999935]

# 양방향 순환 신경망 (Bidirectional Recurrent Neural Network)

- 양방향 순환 신경망 : 시점 t에서의 출력값을 예측할 때 이전 시점의 데이터뿐만 아니라, 이후 데이터로도 예측할 수 있다는 아이디어
- 영어 빈칸 채우기 문제에 비유
    Exercise is very effective at [        ] belly fat.<br>
    
    1) reducing<br>
    2) increasing<br>
    3) multiplying<br>
- 정답을 찾기 위해서는 이전 단어들만으로는 부족, 이후 단어인 belly fat을 봐야함.
![doc1](https://github.com/tenjumh/GraduateSchool/blob/master/Study/NLP_Natural%20Language%20Processing/image/RNN/rnn11.PNG?raw=True)
- 양방향 RNN은 하나의 출력값을 예측하기 위해 기본적으로 두 개의 메모리 셀을 사용
- 첫번째 메모리 셀은 앞에서 배운 것처럼 "앞 시점의 은닉 상태(Forward States)"를 전달받아 현재의 은닉 상태를 계산
- 두번째 메모리 셀은 앞 시점의 은닉 상태가 아니라 뒤 시점의 은닉 상태(Backward States)를 전달 받아 현재의 은닉 상태를 계산

In [ ]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))

- 양방향 RNN의 은닉층이 두개인 경우 그림과 코드
![doc1](https://github.com/tenjumh/GraduateSchool/blob/master/Study/NLP_Natural%20Language%20Processing/image/RNN/rnn12.PNG?raw=True)

In [ ]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))

- keras RNN에 대한 매우 자세한 설명
https://stackoverflow.com/questions/38714959/understanding-keras-lstms
https://gluon.mxnet.io/chapter05_recurrent-neural-networks/simple-rnn.html